# Reader & Writer
##### Objetivos
1. Leia de arquivos CSV
1. Leia de arquivos JSON
1. Gravar DataFrame em arquivos
1. Gravar DataFrame nas tabelas
1. Gravar DataFrame em uma tabela Delta

##### Métodos
- <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#input-and-output" target="_blank">DataFrameReader</a>: `csv`, `json`, `option`, `schema`
- <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#input-and-output" target="_blank">DataFrameWriter</a>: `mode`, `option`, `parquet`, `format`, `saveAsTable`
- <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.types.StructType.html#pyspark.sql.types.StructType" target="_blank">StructType</a>: `toDDL`

##### Tipos Spark
- <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#data-types" target="_blank">Types</a>: `ArrayType`, `DoubleType`, `IntegerType`, `LongType`, `StringType`, `StructType`, `StructField`

## DataFrameReader

Interface usada para carregar um DataFrame de sistemas de armazenamento externos

```
spark.read.parquet("caminho/para/arquivos")
```

DataFrameReader é acessível através do atributo SparkSession `read`. Esta classe inclui métodos para carregar DataFrames de diferentes sistemas de armazenamento externo.

### Ler de arquivos CSV

Leia do CSV com o método `csv` do DataFrameReader e as seguintes opções:

Separador de tabulação, use a primeira linha como cabeçalho, infira o esquema

In [ ]:
usersCsvPath = "dbfs:/FileStore/shared_uploads/jorge.diaz@inf.ufrgs.br/users.csv"

usersDF = (
    spark.read.format("csv")
    .option("header", True)
    .option("sep", ",")
    .load(usersCsvPath)
)

In [ ]:
usersDF.display()

In [ ]:
usersDF.printSchema()

A API Python do Spark também permite que você especifique as opções DataFrameReader como parâmetros para o método `csv`

In [ ]:
usersDF = spark.read.csv(usersCsvPath, sep=",", inferSchema=True, header=True)

usersDF.printSchema()

Defina manualmente o esquema criando um `StructType` com nomes de colunas e tipos de dados e leia dados mais rapidamente

In [ ]:
from pyspark.sql.types import LongType, StringType, StructType, StructField

userDefinedSchema = StructType(
    [
        StructField("user_id", StringType()),
        StructField("user_first_touch_timestamp", LongType()),
        StructField("email", StringType()),
    ]
)

Leia de CSV usando este esquema definido pelo usuário em vez de inferir o esquema

In [ ]:
usersDF = spark.read.csv(usersCsvPath, sep=",", schema=userDefinedSchema, header=True)

usersDF.printSchema()

Como alternativa, defina o esquema usando a sintaxe <a href="https://en.wikipedia.org/wiki/Data_definition_language" target="_blank">linguagem de definição de dados (DDL)</a>.

In [ ]:
DDLSchema = "user_id string, user_first_touch_timestamp long, email string"

usersDF = (
    spark.read.format("csv")
    .option("header", True)
    .option("sep", ",")
    .schema(DDLSchema)
    .load(usersCsvPath)
)

## DataFrameWriter
Interface usada para gravar um DataFrame em sistemas de armazenamento externo

```
(df.write                         
  .option("compression", "snappy")
  .mode("overwrite")      
  .parquet(outPath)       
)
```

DataFrameWriter é acessível por meio do atributo SparkSession `write`. Esta classe inclui métodos para gravar DataFrames em diferentes sistemas de armazenamento externo.

### Gravar DataFrames em arquivos

Escreva `usersDF` no parquet com o método `parquet` do DataFrameWriter e as seguintes configurações:

Compressão `snappy`, modo `overwrite`

In [ ]:
usersOutputPath = "dbfs:/FileStore/shared_uploads/jchambyd@gmail.com/" + "users.parquet"

(
    usersDF.write
    .mode("overwrite")
    .parquet(usersOutputPath)
)

In [ ]:
usersDF2 = spark.read.parquet(usersOutputPath)

display(usersDF2)

In [ ]:
display(
    dbutils.fs.ls(usersOutputPath)
)

Assim como o DataFrameReader, a API Python do Spark também permite que você especifique as opções do DataFrameWriter como parâmetros para o método `parquet`

In [ ]:
(usersDF.write.parquet(usersOutputPath, compression="snappy", mode="overwrite"))

### Gravar DataFrames em tabelas

Escreva `eventsDF` em uma tabela usando o método DataFrameWriter `saveAsTable`

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> Isso cria uma tabela global, ao contrário da visão local criada pelo método DataFrame `createOrReplaceTempView`

In [ ]:
usersDF.write.mode('overwrite').saveAsTable("users")

In [ ]:
usersDF.createOrReplaceTempView("users1")

In [ ]:
%sql

select * from users

## Delta Lake

Em quase todos os casos, a prática recomendada é usar o formato Delta Lake, especialmente sempre que os dados forem referenciados de um espaço de trabalho do Databricks.

<a href="https://delta.io/" target="_blank">Delta Lake</a> é uma tecnologia de código aberto projetada para funcionar com o Spark para trazer confiabilidade aos data lakes.

![delta](https://files.training.databricks.com/images/aspwd/delta_storage_layer.png)

#### Principais recursos do Delta Lake
- Transações ACID
- Handline de metadados escalável
- Streaming unificado e processamento em lote
- Viagem no tempo (versionamento de dados)
- Aplicação e evolução do esquema
- Histórico de auditoria
- Formato parquet
- Compatível com Apache Spark API

### Gravar resultados em uma tabela delta

Escreva `eventsDF` com o método `save` do DataFrameWriter e as seguintes configurações: formato `delta`, modo `overwrite`

In [ ]:
usersOutputPath = "dbfs:/FileStore/shared_uploads/jchambyd@gmail.com/" + "/delta/users"

(usersDF.write.format("delta").mode("overwrite").save(usersOutputPath) )

In [ ]:
usersDFDelta = (
    spark.read.format("delta")
    .load(usersOutputPath)
)

usersDFDelta.display()

# Laboratório de dados de ingestão

Leia em arquivos CSV contendo dados de produtos.

##### Tarefas
1. Leia com esquema de inferência
2. Leia com esquema definido pelo usuário
3. Leia com esquema como string formatada em DDL
4. Escreva usando o formato Delta